In [1]:
!pip install numpy==1.26.0


   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
    --------------------------------------- 0.3/15.5 MB ? eta -:--:--
   -- ------------------------------------- 1.0/15.5 MB 3.4 MB/s eta 0:00:05
   ---- ----------------------------------- 1.8/15.5 MB 3.2 MB/s eta 0:00:05
   ------ --------------------------------- 2.6/15.5 MB 3.3 MB/s eta 0:00:04
   -------- ------------------------------- 3.1/15.5 MB 3.1 MB/s eta 0:00:05
   -------- ------------------------------- 3.4/15.5 MB 2.8 MB/s eta 0:00:05
   ---------- ----------------------------- 3.9/15.5 MB 2.6 MB/s eta 0:00:05
   ----------- ---------------------------- 4.5/15.5 MB 2.6 MB/s eta 0:00:05
   ------------ --------------------------- 4.7/15.5 MB 2.4 MB/s eta 0:00:05
   ------------ --------------------------- 5.0/15.5 MB 2.3 MB/s eta 0:00:05
   -------------- ------------------------- 5.8/15.5 MB 2.4 MB/s eta 0:00:04
   ----------------- ---------------------- 6.8/15.5 MB 2.6 MB/s eta 0:00:04
   ----------

In [2]:
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=70)

c:\Users\FARAZ\Desktop\Projects\Computer-Vision\deep_sort\deep\feature_extractor.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map

In [3]:
import cv2
import torch

# Define the video path
video_path = 'vid.mp4'

cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
frames = []

unique_track_ids = set()

In [11]:
import time
import numpy as np
from ultralytics import YOLO

i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        
        og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = og_frame.copy()

        model = YOLO("runs/detect/train15/weights/best.pt")  # load a pretrained model (recommended for training)

        results = model.predict(source=frame, device='cpu', classes=0, conf=0.8)

        class_names = ['person']

        for result in results:
            boxes = result.boxes  # Boxes object for bbox outputs
            probs = result.probs  # Class probabilities for classification outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            for class_index in cls:
                class_name = class_names[int(class_index)]
                #print("Class:", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)
        
        tracks = tracker.update(bboxes_xywh, conf, og_frame)
        
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
            w = x2 - x1  # Calculate width
            h = y2 - y1  # Calculate height

            # Set color values for red, blue, and green
            red_color = (0, 0, 255)  # (B, G, R)
            blue_color = (255, 0, 0)  # (B, G, R)
            green_color = (0, 255, 0)  # (B, G, R)

            # Determine color based on track_id
            color_id = track_id % 3
            if color_id == 0:
                color = red_color
            elif color_id == 1:
                color = blue_color
            else:
                color = green_color

            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

            text_color = (0, 0, 0)  # Black color for text
            cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            # Add the track_id to the set of unique track IDs
            unique_track_ids.add(track_id)

        # Update the person count based on the number of unique track IDs
        person_count = len(unique_track_ids)

        # Update FPS and place on frame
        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time

        # Draw person count on frame
        cv2.putText(og_frame, f"Person Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Append the frame to the list
        frames.append(og_frame)

        # Write the frame to the output video file
        out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))

        # Show the frame
        #cv2.imshow("Video", og_frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 233.0ms
Speed: 15.0ms preprocess, 233.0ms inference, 12.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 173.0ms
Speed: 7.0ms preprocess, 173.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 558.8ms
Speed: 5.0ms preprocess, 558.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 89.0ms
Speed: 3.0ms preprocess, 89.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 161.0ms
Speed: 3.0ms preprocess, 161.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 125.0ms
Speed: 4.0ms preprocess, 125.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 118.0ms
Speed: 3.0ms preprocess, 118.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 103.0ms
Speed: 4.0ms prepr

KeyboardInterrupt: 

In [3]:
import os

video_path = 'test/MOT16-01/img1/'

# Check if the path exists and contains files
if os.path.isdir(video_path):
    print(f"Path exists: {video_path}")
    files = os.listdir(video_path)
    if files:
        print(f"Files in the path: {files[:5]}")  # Print first 5 files for verification
    else:
        print("The folder is empty.")
else:
    print("Path does not exist.")


Path exists: test/MOT16-01/img1/
Files in the path: ['000001.jpg', '000002.jpg', '000003.jpg', '000004.jpg', '000005.jpg']


In [12]:
import cv2
import torch
import time
import os
import numpy as np
from ultralytics import YOLO
from deep_sort.deep_sort.tracker import Tracker
from deep_sort.deep_sort.nn_matching import NearestNeighborDistanceMetric

In [13]:
# Define image path
image_path = 'test/MOT16-01/img1/'

# Get list of image files
image_files = sorted(os.listdir(image_path))

# Initialize device (GPU/CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Initialize YOLO model
model = YOLO("runs/detect/train15/weights/best.pt")  # Load pretrained YOLO model

# Initialize DeepSORT with TensorFlow model
deep_sort_weights = 'deep_sort/resources/networks/mars-small128.pb'  # Path to TensorFlow model

# Initialize the metric with cosine distance
metric = NearestNeighborDistanceMetric("cosine", 0.7, None)

# Create the tracker with the metric
tracker = Tracker(metric=metric, max_age=70, n_init=3)

# Read the first image to get the width and height
frame = cv2.imread(os.path.join(image_path, image_files[0]))
frame_height, frame_width, _ = frame.shape

# Set FPS manually (since it's images, not a video stream)
fps = 30  # You can adjust this as needed

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_path = 'output4.avi'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))


In [14]:
from deep_sort.deep_sort.detection import Detection

for image_file in image_files:
    img_path = os.path.join(image_path, image_file)
    frame = cv2.imread(img_path)

    if frame is not None:
        og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB for YOLO
        frame = og_frame.copy()

        # Perform object detection
        results = model.predict(source=frame, device=device, classes=0, conf=0.5)
        
        # Check if any boxes were detected
        if len(results[0].boxes) == 0:
            print("No objects detected in this frame")
        else:
            print(f"Detected {len(results[0].boxes)} objects")

        # Extract bounding boxes and confidence values
        conf = []
        xywh = []
        features = []

        for result in results:
            boxes = result.boxes
            xywh.append(boxes.xywh.cpu().numpy())  # Append all box coordinates
            conf.extend(boxes.conf.tolist())  # Append confidence values
            features.append(result.features.cpu().numpy())

        # Flatten the xywh list if needed (if it's a list of lists)
        xywh = np.concatenate(xywh, axis=0)
        features = np.concatenate(features, axis=0)

        if len(xywh) == 0:
            continue  # Skip this frame if no detections
        
        # Create list of Detection objects
        detections = []
        for box, c in zip(xywh, conf):
            detection = Detection(box, c)  # Create Detection object
            detections.append(detection)

        # Update tracker with bounding boxes and confidences
        tracks = tracker.update(detections)

        # Loop through tracked objects and draw bounding boxes
        for track in tracker.tracker.tracks:
            track_id = track.track_id
            x1, y1, x2, y2 = track.to_tlbr()  # Convert to top-left/bottom-right format
            color = (0, 255, 0)  # Green for bounding boxes
            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            cv2.putText(og_frame, f"ID-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

        # Write the frame to the output video
        out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))

# Release the VideoWriter and cleanup
out.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 211.2ms
Speed: 13.2ms preprocess, 211.2ms inference, 22.8ms postprocess per image at shape (1, 3, 384, 640)
Detected 1 objects


AttributeError: 'Results' object has no attribute 'features'. See valid attributes below.

    A class for storing and manipulating inference results.

    This class encapsulates the functionality for handling detection, segmentation, pose estimation,
    and classification results from YOLO models.

    Attributes:
        orig_img (numpy.ndarray): Original image as a numpy array.
        orig_shape (Tuple[int, int]): Original image shape in (height, width) format.
        boxes (Boxes | None): Object containing detection bounding boxes.
        masks (Masks | None): Object containing detection masks.
        probs (Probs | None): Object containing class probabilities for classification tasks.
        keypoints (Keypoints | None): Object containing detected keypoints for each object.
        obb (OBB | None): Object containing oriented bounding boxes.
        speed (Dict[str, float | None]): Dictionary of preprocess, inference, and postprocess speeds.
        names (Dict[int, str]): 

In [28]:
import cv2

# Open the video file
video_path = 'output3.avi'
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    # Loop to read frames
    while True:
        ret, frame = cap.read()
        
        # If frame is read correctly
        if ret:
            cv2.imshow('Video Playback', frame)
            
            # Break on key press (e.g., pressing 'q' to quit)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()


In [33]:
!python deep_sort/deep_sort_app2.py --sequence_dir=test/MOT16-03 --detection_file=deep_sort/resources/detections/MOT16_POI_test/MOT16-03.npy --min_confidence=0.3 --nn_budget=100 --display=True

Processing frame 00001

0: 384x640 44 persons, 95.6ms
Speed: 4.8ms preprocess, 95.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Traceback (most recent call last):
  File "c:\Users\FARAZ\Desktop\Projects\Computer-Vision\deep_sort\deep_sort_app2.py", line 125, in <module>
    run(
  File "c:\Users\FARAZ\Desktop\Projects\Computer-Vision\deep_sort\deep_sort_app2.py", line 102, in run
    visualizer.run(frame_callback)
  File "c:\Users\FARAZ\Desktop\Projects\Computer-Vision\deep_sort\application_util\visualization.py", line 100, in run
    self.viewer.run(lambda: self._update_fun(frame_callback))
  File "c:\Users\FARAZ\Desktop\Projects\Computer-Vision\deep_sort\application_util\image_viewer.py", line 305, in run
    self._terminate = not self._user_fun()
                          ^^^^^^^^^^^^^^^^
  File "c:\Users\FARAZ\Desktop\Projects\Computer-Vision\deep_sort\application_util\visualization.py", line 100, in <lambda>
    self.viewer.run(lambda: self._update_fun(frame_callback))
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\FARAZ\Desktop\Projects\Computer-Vision\deep_sort\application_